In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/Raflinfp/TugasSistemRekomendasi/main/tgssistemrekomendasi.csv")
df.head()

,Name,Address,Description
0,Rafli Nur Fadhillah Panca,"Gisik Sari RT4 RW 4 Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,..."
1,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Akademi Digital Marketing. CRM. Web Design unt...
2,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Android Java for Mobile Developer. User Interf...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Frontend Engineering. Final Project. Programmi...
4,PT Mitra Semeru Indonesia,"Jl. Mampang Prapatan Raya no 84A, Tegal Parang...",Perancangan Produk/Jasa. Komunikasi. Leadershi...


In [6]:
df.describe()

,Name,Address,Description
count,11,11,11
unique,11,11,11
top,Rafli Nur Fadhillah Panca,"Gisik Sari RT4 RW 4 Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,..."
freq,1,1,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         11 non-null     object
 1   Address      11 non-null     object
 2   Description  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


In [8]:
def print_description(index):
    example = df[df.index == index][['Description', 'Name','Address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
        print('Address:', example[2])

In [9]:
print_description(1)

Akademi Digital Marketing. CRM. Web Design untuk Digital Marketing. Content Marketing. Social Media Marketing. Strategi Digital Marketing. SEO dan Web Analystics. Graphics Design untuk Digital Marketing. Pengantar Digital Marketing.
Name: PT Nurul Fikri Cipta Inovasi 
Address: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu,  Kec. Cimanggis, Kota Depok, Jawa Barat


In [10]:
print_description(3)

Frontend Engineering. Final Project. Programming JavaScript. Software Engineering Frontend Web Development With HTML & CSS. Interacting With Web APIs. Intro To Node.js.
Name: PT Ruang Raya Indonesia
Address: Jl. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [11]:
print_description(5)

Pengembang Aplikasi Android. Pengenalan Logika Pemrograman. Belajar Dasar Github. Pemrograman Kotlin. Fundamental aplikasi Android. Pengembangan Aplikasi Android Intermediate. Prinsip Pemrograman SOLID. Dasar UX Design. 
Name: PT Dicoding Akademi Indonesia 
Address: Jalan Batik Kumeli No 50, Kecamatan : Cibeunying Kaler, Kelurahan : Sukaluyu, RT : 10, RW : 07, Kota Bandung, Jawa Barat, 40123


In [12]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['Description'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
df.head()

,Name,Address,Description,desc_clean
0,Rafli Nur Fadhillah Panca,"Gisik Sari RT4 RW 4 Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,...",algoritma dan pemrograman bahasa pemrograman ...
1,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Akademi Digital Marketing. CRM. Web Design unt...,akademi digital marketing crm web design untuk...
2,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Android Java for Mobile Developer. User Interf...,android java for mobile developer user interfa...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Frontend Engineering. Final Project. Programmi...,frontend engineering final project programming...
4,PT Mitra Semeru Indonesia,"Jl. Mampang Prapatan Raya no 84A, Tegal Parang...",Perancangan Produk/Jasa. Komunikasi. Leadershi...,perancangan produk jasa komunikasi leadership ...


In [14]:
def print_description_clean(index):
    example = df[df.index == index][['desc_clean', 'Name','Address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan:', example[1])
        print('Alamat:', example[2])

In [15]:
print_description_clean(1)

akademi digital marketing crm web design untuk digital marketing content marketing social media marketing strategi digital marketing seo dan web analystics graphics design untuk digital marketing pengantar digital marketing
Nama Perusahaan: PT Nurul Fikri Cipta Inovasi 
Alamat: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu,  Kec. Cimanggis, Kota Depok, Jawa Barat


In [16]:
print_description_clean(3)

frontend engineering final project programming javascript software engineering frontend web development with html  css interacting with web apis intro to nodejs
Nama Perusahaan: PT Ruang Raya Indonesia
Alamat: Jl. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [17]:
print_description_clean(5)

pengembang aplikasi android pengenalan logika pemrograman belajar dasar github pemrograman kotlin fundamental aplikasi android pengembangan aplikasi android intermediate prinsip pemrograman solid dasar ux design 
Nama Perusahaan: PT Dicoding Akademi Indonesia 
Alamat: Jalan Batik Kumeli No 50, Kecamatan : Cibeunying Kaler, Kelurahan : Sukaluyu, RT : 10, RW : 07, Kota Bandung, Jawa Barat, 40123


In [18]:
df.set_index('Name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.04070524, 0.0108891 , 0.02960136, 0.02881204,
        0.11906387, 0.01620249, 0.        , 0.        , 0.05972098,
        0.05639714],
       [0.04070524, 1.        , 0.        , 0.02384393, 0.05921582,
        0.00762089, 0.        , 0.03742761, 0.01303843, 0.        ,
        0.00499699],
       [0.0108891 , 0.        , 1.        , 0.01139597, 0.        ,
        0.05472744, 0.        , 0.        , 0.        , 0.        ,
        0.00955305],
       [0.02960136, 0.02384393, 0.01139597, 1.        , 0.        ,
        0.        , 0.01096213, 0.        , 0.        , 0.        ,
        0.        ],
       [0.02881204, 0.05921582, 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.11906387, 0.00762089, 0.05472744, 0.        , 0.        ,
        1.        , 0.03509599, 0.0061228 , 0.0090392 , 0.02308856,
        0.        ],
       [0.01620249, 0.        , 0.        , 0.01096213, 0.

In [19]:
indices = pd.Series(df.index)
indices[:50]

0                 Rafli Nur Fadhillah Panca
1             PT Nurul Fikri Cipta Inovasi 
2        PT Hacktivate Teknologi Indonesia 
3                   PT Ruang Raya Indonesia
4                 PT Mitra Semeru Indonesia
5            PT Dicoding Akademi Indonesia 
6     PT. IlmuKomputerCom Braindevs Sistema
7         PT Telkom Indonesia (Persero) Tbk
8                   PT Global Tiket Network
9                             PT Berau Coal
10                     PT Pundi Mas Berjaya
Name: Name, dtype: object

In [20]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

In [21]:
recommendations('Rafli Nur Fadhillah Panca')

['PT Dicoding Akademi Indonesia ', 'PT Berau Coal', 'PT Pundi Mas Berjaya']

In [22]:
recommendations('PT Mitra Semeru Indonesia')

['PT Nurul Fikri Cipta Inovasi ',
 'Rafli Nur Fadhillah Panca',
 'PT Hacktivate Teknologi Indonesia ']

In [25]:
recommendations('PT Global Tiket Network')

['PT. IlmuKomputerCom Braindevs Sistema',
 'PT Nurul Fikri Cipta Inovasi ',
 'PT Telkom Indonesia (Persero) Tbk']